In [260]:
#import library
import pandas as pd
import numpy as np

In [261]:
#create dataframe
data = {'ID':['001','002','003','004','003','005'],'Order':['Buy','Sell','Buy','Buy','Buy','Sell'],
        'Type':['Add','Add','Add','Add','Remove','Add'],'Price':[20,25,23,23,23,28],'Quantity':[100,200,50,70,50,100]}
df = pd.DataFrame(data)
df

,ID,Order,Type,Price,Quantity
0,001,Buy,Add,20,100
1,002,Sell,Add,25,200
2,003,Buy,Add,23,50
3,004,Buy,Add,23,70
4,003,Buy,Remove,23,50
5,005,Sell,Add,28,100


In [262]:
#Adding a removing operation
def Remove(ID):
    global df #Find the dataframe as a global variable
    if len(df[(df['ID']==ID) & (df['Type']=='Add')]) != 1: #if there's 0 or 2 instance of an ID, it's not an ID to "Remove"
        print('ID not correct')
    else:
        NewRow = df[df['ID']==ID].replace(to_replace='Add',value='Remove') #Find the row with this ID and invert 'Type'
        df = pd.concat([df,NewRow]) #Add new row into the dataframe

In [263]:
#Example by "Removing" ID 005
Remove('005')
df

,ID,Order,Type,Price,Quantity
0,001,Buy,Add,20,100
1,002,Sell,Add,25,200
2,003,Buy,Add,23,50
3,004,Buy,Add,23,70
4,003,Buy,Remove,23,50
5,005,Sell,Add,28,100
5,005,Sell,Remove,28,100


In [264]:
#Adding an adding operation
def Add(Order,Price,Quantity):
    global df #Find the dataframe as a global variable
    NewID = '0'*(3-len(str(int(df['ID'].max()))))+str(int(df['ID'].max())+1) #Find a new biggest ID, and make it match the three digit format
    df = df._append({'ID':NewID,'Order':Order,'Type':'Add','Price':Price,'Quantity':Quantity},ignore_index=True) #Add new row

In [265]:
#Adding new values
Add('Buy',24,50)
Add('Sell',20,10)
df

,ID,Order,Type,Price,Quantity
0,001,Buy,Add,20,100
1,002,Sell,Add,25,200
2,003,Buy,Add,23,50
3,004,Buy,Add,23,70
4,003,Buy,Remove,23,50
5,005,Sell,Add,28,100
6,005,Sell,Remove,28,100
7,006,Buy,Add,24,50
8,007,Sell,Add,20,10


In [266]:
#Finding best offers operation
def Find(Order,DesiredQuantity):
    Temp = df[~df['ID'].isin(df[df['Type']=='Remove']['ID'])] #Temporary dataframe without any "Removed" rows
    if Order == 'Buy':
        SellOrders = Temp[Temp['Order']=='Sell'].sort_values('Price') #Sort values by lowest price
        ToFind = len(SellOrders['Quantity'][SellOrders['Quantity'].cumsum()<DesiredQuantity])+1 #Find how many rows need to be returned to satisfy DesiredQuantity
        return(SellOrders[:ToFind])
    if Order == 'Sell':
        BuyOrders = Temp[Temp['Order']=='Buy'].sort_values('Price',ascending = False) #Sort values by highest price
        ToFind = len(BuyOrders['Quantity'][BuyOrders['Quantity'].cumsum()<DesiredQuantity])+1 #Find how many rows need to be returned to satisfy DesiredQuantity
        return(BuyOrders[:ToFind])

In [267]:
Find('Buy',100)

,ID,Order,Type,Price,Quantity
8,007,Sell,Add,20,10
1,002,Sell,Add,25,200


100 units can be sold if we buy the 10, and the remaining 90 from next best sell order

In [268]:
Find('Sell',300)

,ID,Order,Type,Price,Quantity
7,006,Buy,Add,24,50
3,004,Buy,Add,23,70
0,001,Buy,Add,20,100


There's not enough buy orders to cover our desired 300 units, so all offers are displayed